<a href="https://colab.research.google.com/github/ZonaQuant/Notebooks/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock screener using Yahoo Finance

The easiest way to make your own stock screener.

You can add, remove or change the different filters.

Additionally, you can add:

* Email alerts
* Twitter alerts

**NOTE: You should adapt that script to download/read files from Google Drive. Alternatively, you can download this script and execute it in your local environment.**

I hope that you enjoy it and remember, you can follow me on Twitter for more like this 😉 

👉 https://twitter.com/justoherrero20

## Imports

In [ ]:
from typing import List
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time

yf.pdr_override()

## Functions definition

In [ ]:
def get_tickers(index_name: str) -> List:
    """
    Given an index from the following list, returns all the tickers.
    * NASDAQ
    * SP500
    * DOW

    Parameters
    ----------
    index: string
        Selected index

    Returns
    -------
    tickers: List
        Contains all the tickers of the selected index
    """
    index = {
        'SP500': si.tickers_sp500(),
        'NASDAQ': si.tickers_nasdaq(),
        'DOW': si.tickers_dow()
    }

    tickers = index.get(index_name)
    # YF uses dashes instead of dots
    return [item.replace(".", "-") for item in tickers]

def get_index_return(index: str) -> float:
    """
    """
    start_date = datetime.datetime.now() - datetime.timedelta(days=365)
    end_date = datetime.date.today()

    index_df = pdr.get_data_yahoo(index, start_date, end_date)
    index_df['Percent Change'] = index_df['Adj Close'].pct_change()
    return (index_df['Percent Change'] + 1).cumprod()[-1]

def download_historical_csv_multiples(tickers: str, index_return: float) -> List:
    """
    Given a ticker, downloads it historical data (prices)
    and export it to a csv file

    Parameters
    ----------
    tickers: list
        Ticker list of an index
    index_return: float
        Index return

    Returns
    -------
    """
    returns_multiples = []
    end_date = datetime.date.today()
    start_date = datetime.datetime.now() - datetime.timedelta(days=365)
    
    for ticker in tickers:
        df = pd.read_csv(f'tickers/{ticker}.csv', index_col=0)
        if df.empty:
            df = pdr.get_data_yahoo(ticker, start_date, end_date)
            df.to_csv(f'tickers/{ticker}.csv')
            time.sleep(1)
        df['Percent Change'] = df['Adj Close'].pct_change()
        stock_return = (df['Percent Change'] + 1).cumprod()[-1]
        
        returns_multiple = round((stock_return / index_return), 2)
        returns_multiples.extend([returns_multiple])
        
        #print (f'Ticker: {ticker}; Returns Multiple against S&P 500: {returns_multiple}\n')
        

    
    return returns_multiples

def get_upper_quantile(tickers: List, returns_multiples: List, quantile: float) -> pd.DataFrame:
    """
    """
    rs_df = pd.DataFrame(list(zip(tickers, returns_multiples)), 
                         columns=['Ticker', 'Returns_multiple'])
    rs_df['RS_Rating'] = rs_df.Returns_multiple.rank(pct=True) * 100
    rs_df = rs_df[rs_df.RS_Rating >= rs_df.RS_Rating.quantile(quantile)]
    return rs_df

def screener(rs_df: pd.DataFrame) -> pd.DataFrame:
    """
    

    Parameters
    ----------

    Returns
    -------
    """
    exportList = pd.DataFrame(columns=['Stock', 'RS_Rating', '50 Day MA',
                                       '150 Day Ma', "200 Day MA", '52 Week Low',
                                       '52 week High'])
    stock_list = rs_df['Ticker']
    sma = [10, 21, 50, 100, 150, 200]

    for stock in stock_list:
        try:
            df = pd.read_csv(f'tickers/{stock}.csv', index_col=0)
            
            for x in sma:
                df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(), 2)
            
            # Storing required values 
            currentClose = df["Adj Close"][-1]
            moving_average_50 = df["SMA_50"][-1]
            moving_average_150 = df["SMA_150"][-1]
            moving_average_200 = df["SMA_200"][-1]
            low_of_52week = round(min(df["Low"][-260:]), 2)
            high_of_52week = round(max(df["High"][-260:]), 2)
            RS_Rating = round(rs_df[rs_df['Ticker']==stock].RS_Rating.tolist()[0])
            
            try:
                moving_average_200_20 = df["SMA_200"][-20]
            except Exception:
                moving_average_200_20 = 0

            conditions = {
                'condition1': currentClose > moving_average_150 > moving_average_200,
                'condition2': moving_average_150 > moving_average_200,
                'condition3': moving_average_200 > moving_average_200_20,
                'condition4': moving_average_50 > moving_average_150 > moving_average_200,
                'condition5': currentClose > moving_average_50,
                'condition6': currentClose >= (1.3*low_of_52week),
                'condition7': currentClose >= (.75*high_of_52week)
            }
            conditions['count_conditions'] = sum(conditions.values())

            """
            if all(conditions.values()):
                exportList = exportList.concat({'Stock': stock, 'RS_Rating': RS_Rating,
                                                '50 Day MA': moving_average_50, 
                                                '150 Day Ma': moving_average_150, 
                                                '200 Day MA': moving_average_200, 
                                                '52 Week Low': low_of_52week, 
                                                '52 week High': high_of_52week}, 
                                                ignore_index=True)
                print(f'{stock} made the Minervini requirements')
            """
            exportList = exportList.append({'Stock': stock, 'RS_Rating': RS_Rating,
                                                '50 Day MA': moving_average_50, 
                                                '150 Day Ma': moving_average_150, 
                                                '200 Day MA': moving_average_200, 
                                                '52 Week Low': low_of_52week, 
                                                '52 week High': high_of_52week,
                                                'Total Conditions': conditions['count_conditions']}, 
                                                ignore_index=True)
                
        except Exception as e:
            print (e)
            print(f"Could not retrieve data on {stock}")
    print(exportList.sort_values(by='RS_Rating', ascending=False))
    return exportList.sort_values(by='RS_Rating', ascending=False)

def to_excel(data: pd.DataFrame, filename:str ='ScreenOutput.xlsx') -> None:
    """
    Writes a dataframe in an Excel file.

    Parameters
    ----------
    data: pd.DataFrame
        Dataframe with information to be written
    filename: str
        Contains the path or name of the file

    Returns
    -------
    None
    """
    writer = ExcelWriter(filename)
    data.to_excel(writer, "Sheet1")
    writer.save()

def main() -> None:
    """
    Main program.

    Parameters
    ----------
    None

    Returns
    -------
    None
    """
    tickers = get_tickers('SP500')
    index_return = get_index_return('^GSPC')
    returns_multiples = download_historical_csv_multiples(tickers, index_return)
    rs_df = get_upper_quantile(tickers, returns_multiples, .70)
    screner_data = screener(rs_df)
    to_excel(screner_data)


## Main Program Execution

In [ ]:
main()

[*********************100%***********************]  1 of 1 completed


/var/folders/00/9cyx9pnx3wv3sk908qkwpfkc0000gn/T/ipykernel_1470/2274753216.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exportList = exportList.append({'Stock': stock, 'RS_Rating': RS_Rating,
/var/folders/00/9cyx9pnx3wv3sk908qkwpfkc0000gn/T/ipykernel_1470/2274753216.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exportList = exportList.append({'Stock': stock, 'RS_Rating': RS_Rating,
/var/folders/00/9cyx9pnx3wv3sk908qkwpfkc0000gn/T/ipykernel_1470/2274753216.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exportList = exportList.append({'Stock': stock, 'RS_Rating': RS_Rating,
/var/folders/00/9cyx9pnx3wv3sk908qkwpfkc0000gn/T/ipykernel_1470/2274753216.py:143: FutureWarning: The frame.append method is depre

    Stock RS_Rating 50 Day MA 150 Day Ma 200 Day MA 52 Week Low 52 week High  \
51    DVN       100     56.34      45.49      40.64       20.14        63.26   
31     CF       100     88.61      70.36      64.45       43.19        110.4   
17    APA       100     37.24      29.94      27.13       15.55        43.88   
94    MOS        99     56.35      43.99      40.78       28.26        74.75   
39    COP        99      96.0      79.97      73.89       47.85       107.52   
..    ...       ...       ...        ...        ...         ...          ...   
27    BXP        71    121.43     116.22     115.32       103.2       133.11   
35    CME        71    238.81     221.19     216.23      185.79       256.94   
124   RMD        71    243.28      253.9     257.24      187.09       301.34   
109   PEG        71     66.85      64.06       63.3       58.96        72.64   
32    CHD        71     98.92      93.84      91.53       80.76       104.84   

     Total Conditions  
51             

/var/folders/00/9cyx9pnx3wv3sk908qkwpfkc0000gn/T/ipykernel_1470/2274753216.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exportList = exportList.append({'Stock': stock, 'RS_Rating': RS_Rating,
